### 각 칼럼별 unique value 확인
* 칼럼별로 어떤 value를 가지고 있는 지
* 그리고 각 비중(%) 어떻게 되는 지 확인

In [ ]:
# count 파라미터는 unique value를 몇 개까지만 볼꺼냐?
# 예) Sex는 남, 여 2개
# 예2) 연령대는 10대, 20대, ~ , 60대 6개이므로 count가 5일때는 표시되지 않음

def col_value_check(df,count) :
    for i in df.columns :
        print("{}'s Nunique: {}\n".format(i,df[i].nunique()))

        if df[i].nunique() < count :
            print("{:=^20}".format(i))
            print(round(df[i].value_counts() / df[i].value_counts().sum(),2))
            print("{:=^20}".format("="))
    return
col_value_check(test,5)

### NaN Table
* 적용 상황
    - train, test의 각 칼럼별 NaN 값을 확인하고자 할 때


In [ ]:
# train 및 test에서 각각의 na 값을 DF형태로 반환

def make_na_table(train, test):
    train_na = train.isna().sum()
    test_na = test.isna().sum()
    na_table = pd.concat((train_na, test_na), join='outer', axis=1, sort=False, keys=('train','test'))
    return na_table
na_table = make_na_table(train,test)
na_table

* kaggle에서 발췌한 missing value df 만들기 함수

In [ ]:
def check_missing_data(df):
    flag=df.isna().sum().any()
    if flag==True:
        total = df.isnull().sum()
        percent = (df.isnull().sum())/(df.isnull().count()*100)
        output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        data_type = []
        # written by MJ Bahmani
        for col in df.columns:
            dtype = str(df[col].dtype)
            data_type.append(dtype)
        output['Types'] = data_type
        return(np.transpose(output))
    else:
        return(False)

### Unique value Table
* 적용 상황
    - train, test의 각 칼럼별 unique value을 확인하고자 할 때
    - columns이 index로 들어감


In [ ]:
def make_unique_value_table(train, test) :
    a = []
    b = []

    for x in range(0, len(train.columns)) :
        a.append(train[train.columns[x]].nunique())
        b.append(train.columns[x])
    a1 = pd.Series(a, index=b)

    c = []
    d = []
    for x in range(0, len(test.columns)) :
        c.append(int(test[test.columns[x]].nunique()))
        d.append(test.columns[x])

    a2 = pd.Series(c, index=d)

    unique_table = pd.concat([a1,a2], axis=1, sort=False, keys=['train', 'test'])
    return unique_table
unique_table = make_unique_value_table(train, test)
unique_table

### IQR 로 Outlier 찾아내는 함수

In [ ]:
# Outlier 찾아내는 함수
def get_outlier(df, col, weight=1.5) :
    col = df[col]
    quan_25 = np.percentile(col.values, 25)
    quan_75 = np.percentile(col.values, 75)
    
    # IQR을 구하고, 1.5를 곱해서 최대/최소를 구함
    iqr = quan_75 - quan_25
    iqr_weight = iqr * weight
    min_val = quan_25 - iqr_weight
    max_val = quan_75 + iqr_weight
    outlier_index = col[(col < min_val) | (col > max_val)].index
    return outlier_index

# 사용 예제

for i in ride_col :
    outlier_index = get_outlier(train2, i)
    print("{}: {}".format(i,train2.loc[outlier_index][i].shape[0]))
print()
for i in off_col :
    outlier_index = get_outlier(train2, i)
    print("{}: {}".format(i,train2.loc[outlier_index][i].shape[0]))
print()

outlier_index = get_outlier(train2, '18~20_ride')
print("{}: {}".format('18~20_ride',train2.loc[outlier_index]['18~20_ride'].shape[0]))

### 이상치 제거 함수

In [ ]:
# 이상치 제거 함수

def del_outlier(df, col_list) :
    df_copy = df.copy()
    for i in col_list :
        outlier_index = get_outlier(df_copy, i)
        df_copy.drop(outlier_index, axis=0, inplace=True)
        print(i)
    return df_copy

### PCA 및 Randomforest 산출 함수
* 적용 상황
    - PCA를 통해 차원 축소가 필요한 데, 몇 개의 component가 필요한 지 모를 때
    - components 수에 따른 R2(모델설명력)와 Randomforest의 score를 산출함

In [ ]:
# 컬럼 최대개수에 맞게 components 수를 변화시키면서 설명력을 확인함
# 

def pca_rcf(train_x, train_y, test_x, test_y) :
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.decomposition import PCA
    
    max_sum, max_com, max_result = 0.0, 0, 0.0
    final_rcf = RandomForestClassifier()
    
    # components를 2~5개까지 테스트 해봄
    for i in range(2, 5) :
        pca = PCA(n_components=i)
        pca_train_x = pca.fit_transform(train_x)
        imp = []
        
        for p in pca.explained_variance_ratio_ :
            imp.append(round(p,4))
            
        print("components:",i,"\tTot:",sum(imp))
        rcf = RandomForestClassifier(n_estimators=1000, random_state=2442)
        rcf.fit(train_x, train_y)
        predicted = rcf.predict(test_x)
        result = rcf.score(test_x, test_y)
        print("\tScore: {:.3f}".format(result))
        
        if result > max_result :
            max_result = result
            final_rcf = rcf

        if sum(imp) > max_sum :
            max_sum = sum(imp)
            max_com = i
            
    print("최종 >> 컴포넌트수: {}, R2: {:.3f}, 예측 정확도: {:.3f}".format(max_com, max_sum, max_result))
    # return 값으로 rcf 모델과 예측값을 반환함 (필요에 따라 수정혀)
    return final_rcf, predicted

rcf, predicted = pca_rcf(train_x, train_y, test_x, test_y)


### 클러스터별 실루엣(클러스터링 점수) 확인
* 아래 코드는 Kmeans로 클러스터링 한 후, 실루엣을 뽑아보는 것
* 추후 클러스터링 알고리즘별로 제작하는 것도 방법일 듯
* 한계
    1. 실루엣 스코어 만으로 클러스터링의 정확도를 판단하는 것은 무리
    2. 실제 분포(시각화)를 통해서 눈으로 보는 것도 중요함
    3. 여러 클러스터링을 사용하여 최적의 군집개수(k)를 찾아보고 테스트 해보는 것을 추천함

In [ ]:
def visualize_silhouette(cluster_lists, X_features): 
    
    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math
    
    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)
    
    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)
    
    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):
        
        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)
        
        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)
        
        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                          'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])
        
        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()
            
            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i
            
            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10
            
        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")
visualize_silhouette([2,3,4,5,6], df)

<hr>

### 모델 성능 평가

* classification 평가
    * classification report
    * roc_auc
    * confusion matrix

In [ ]:
def get_model_train_eval(model, train_x,  test_x, train_y, test_y) :
    from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
    
    model.fit(train_x, train_y)
    pred = model.predict(test_x)
    print("==Confusion_matrix below==\n",confusion_matrix(test_y, pred))
    print(classification_report(test_y, pred, digits=3))  # digit, 3자리 수까지만 표기
    print()
    print("{:>12}\t{:.3f}".format("ROC-AUC",roc_auc_score(test_y, pred))) 

### GridSearchCV - 교차 검증과 체적 하이퍼 파라미터 튜닝을 한 번에

In [ ]:
from sklearn.model_selection import GridSerchCV

params = {
    "" : [],
    "" : []
}

grid_cv = GridSerchCV(model, param_grid = params, cv = 2, n_jobs = -1)
grid_cv.fit(train_x, train_y)
print("Best Params:\n", grid_cv.best_params_)
print("Best predict score: {:.4f}".format(grid_cv.best_score_))

<hr>

### 전화번호, URL, PRICE 필터링

In [ ]:
def phone_number_filter(text):
    re_pattern = r'\d{2,3}[-\.\s]*\d{3,4}[-\.\s]*\d{4}(?!\d)'
    new_text = re.sub(re_pattern, 'tel', text)
    re_pattern = r'\(\d{3}\)\s*\d{4}[-\.\s]??\d{4}'
    new_text = re.sub(re_pattern, 'tel', new_text)
    return new_text
    
    
def url_filter(text):
    re_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),|]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    new_text = re.sub(re_pattern, 'url', text)
    return new_text


def price_filter(text):
    re_pattern = r'\d{1,3}[,\.]\d{1,3}[만\천]?\s?[원]|\d{1,5}[만\천]?\s?[원]'
    text = re.sub(re_pattern, 'money', text)
    re_pattern = r'[일/이/삼/사/오/육/칠/팔/구/십/백][만\천]\s?[원]'
    text = re.sub(re_pattern, 'money', text)
    re_pattern = r'(?!-)\d{2,4}[0]{2,4}(?!년)(?!.)|\d{1,3}[,/.]\d{3}'
    new_text = re.sub(re_pattern, 'money', text)
    return new_text

<hr>

### 샘플링
* Imbalanced Data Set에 적용
* 모든 알고리즘을 적용한 것은 아님
* [imblearn site](https://imbalanced-learn.readthedocs.io/en/stable/api.html)

In [1]:
def make_sampling(X_train, y_train) :
    '''https://imbalanced-learn.readthedocs.io/en/stable/api.html
    현재 5개만 추가된 상태'''
    
    from imblearn.under_sampling import EditedNearestNeighbours, NeighbourhoodCleaningRule, TomekLinks
    from imblearn.over_sampling import SVMSMOTE
    from imblearn.combine import SMOTEENN
    from imblearn.metrics import sensitivity_score, specificity_score
    
    # Edited Nearest Neighbours (이웃 개수 및 kidn_sel ('all'/ 'mode') 둘 중 한개 택
    x_enn, y_enn = EditedNearestNeighbours(sampling_strategy='auto', n_neighbors=3, 
                                           kind_sel='all', n_jobs=-1).fit_sample(X_train, y_train)
    
    # threshold_cleaing은 임계값
    x_ncr, y_ncr = NeighbourhoodCleaningRule(sampling_strategy='auto', n_neighbors=3,
                                             kind_sel='all', threshold_cleaning=0.5, n_jobs=-1).fit_sample(X_train, y_train)
    
    # SMOTE + ENN : only can use 'binary' 
    x_s_enn, y_s_enn = SMOTEENN(sampling_strategy='auto', random_state=None, n_jobs=-1).fit_sample(X_train, y_train)
    
    # Tomek's llnk
    x_t, y_t = TomekLinks(n_jobs=-1).fit_sample(X_train, y_train)
    
    x_s_s, y_s_s = SVMSMOTE(n_jobs=-1).fit_sample(X_train, y_train)
    
    sampling_list = [x_enn, y_enn, x_ncr, y_ncr, x_s_enn, y_s_enn, x_t, y_t, x_s_s, y_s_s]
    
    return sampling_list

help(make_sampling)

###################

def get_eval(y_test, pred) :
    from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score 
    
    cm = confusion_matrix(y_test, pred)
    acc = np.round(accuracy_score(y_test, pred),3)
    pre = np.round(precision_score(y_test, pred),3)
    recall = np.round(recall_score(y_test, pred),3)
    f1 = np.round(f1_score(y_test, pred),3)
    roc_auc = np.round(roc_auc_score(y_test, pred),3)

    return cm, acc, pre, recall, f1, roc_auc
######################

def make_series(y_test, pred, index_name) :
    cm, acc, pre, recall, f1, roc_auc = get_eval(y_test, pred)
    score = pd.Series({'c_m':cm, 'acc': acc, 'precision':pre, 'recall':recall, 'f1':f1, 'roc_auc':roc_auc},
                             name=index_name)
    return score

Help on function make_sampling in module __main__:

make_sampling(X_train, y_train)
    https://imbalanced-learn.readthedocs.io/en/stable/api.html
    현재 5개만 추가된 상태



In [ ]:
# 위에서 샘플링된 것으로 모델에 넣는 것
# 현재는 LGBM을 기본 모델로 채택

def use_sampling(sampling_list, x_test, y_test) :
    # sampling api 리스트
    api = ['ENN', 'NCR', 'SMOTEENN', 'Tomek', 'SVMSMOTE']
    
    from lightgbm import LGBMClassifier

    lgbm = LGBMClassifier(n_estimators=200, n_jobs=-1, random_state=1, learning_rate=0.1)
    evals = [(x_test, y_test)]
    score_df = pd.DataFrame(columns=['c_m','acc','precision','recall','f1','roc_auc'])
    
    for i in range(0,len(sampling_list),2) :
        lgbm.fit(sampling_list[i], sampling_list[i+1], early_stopping_rounds=50, eval_metric="auc", eval_set=evals, verbose=False)
        pred = lgbm.predict(x_test)
        
        if i != 0 :
            score = make_series(y_test, pred, api[int(i/2)])
            score_df = score_df.append(score)
        else :
            score = make_series(y_test, pred, api[i])
            score_df = score_df.append(score)
    
    return score_df
        

* 실사용 예시 (x_train, y_train, x_test, y_test 모두 있어야함)

In [ ]:
sampling_list = make_sampling(x_train, y_train)
print(len(sampling_list))  # 모델별로 2개씩(x, y) 들어가므로 총 10개가 반환

score_df = use_sampling(sampling_list, x_test, y_test)
score_df